In [2]:
#@title Install dependencies

!pip install -q torchaudio omegaconf

import torch
from pprint import pprint
from omegaconf import OmegaConf
from IPython.display import Audio, display

torch.hub.download_url_to_file('https://raw.githubusercontent.com/snakers4/silero-models/master/models.yml',
                               'latest_silero_models.yml',
                               progress=False)
models = OmegaConf.load('latest_silero_models.yml')

In [3]:
# see latest avaiable models
available_languages = list(models.tts_models.keys())
print(f'Available languages {available_languages}')

for lang in available_languages:
    _models = list(models.tts_models.get(lang).keys())
    print(f'Available models for {lang}: {_models}')

Available languages ['ru', 'en', 'de', 'es', 'fr', 'ba', 'xal', 'tt', 'uz', 'ua', 'indic', 'multi']
Available models for ru: ['v3_1_ru', 'ru_v3', 'aidar_v2', 'aidar_8khz', 'aidar_16khz', 'baya_v2', 'baya_8khz', 'baya_16khz', 'irina_v2', 'irina_8khz', 'irina_16khz', 'kseniya_v2', 'kseniya_8khz', 'kseniya_16khz', 'natasha_v2', 'natasha_8khz', 'natasha_16khz', 'ruslan_v2', 'ruslan_8khz', 'ruslan_16khz']
Available models for en: ['v3_en', 'v3_en_indic', 'lj_v2', 'lj_8khz', 'lj_16khz']
Available models for de: ['v3_de', 'thorsten_v2', 'thorsten_8khz', 'thorsten_16khz']
Available models for es: ['v3_es', 'tux_v2', 'tux_8khz', 'tux_16khz']
Available models for fr: ['v3_fr', 'gilles_v2', 'gilles_8khz', 'gilles_16khz']
Available models for ba: ['aigul_v2']
Available models for xal: ['v3_xal', 'erdni_v2']
Available models for tt: ['v3_tt', 'dilyara_v2']
Available models for uz: ['v3_uz', 'dilnavoz_v2']
Available models for ua: ['v3_ua', 'mykyta_v2']
Available models for indic: ['v3_indic']
Avail

In [4]:
import torch

language = 'ru'
model_id = 'v3_1_ru'
device = torch.device('cpu')

model_tts, example_text = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                     model='silero_tts',
                                     language=language,
                                     speaker=model_id)
model_tts.to(device)  # gpu or cpu

Using cache found in C:\Users\ander/.cache\torch\hub\snakers4_silero-models_master


In [5]:
model_tts.speakers

['aidar', 'baya', 'kseniya', 'xenia', 'eugene', 'random']

In [8]:
sample_rate = 48000
speaker = 'eugene' # Можно сделать вообще рандом, а потом сохранить через model.save_random_voice(voice_path) и model.apply_tts(voice_path=voice_path...)
put_accent=True
put_yo=True

example_text = '''
<speak>
              <p>
                  Когда я просыпаюсь, <prosody rate="x-slow">я говорю довольно медленно</prosody>.
                  Пот+ом я начинаю говорить своим обычным голосом,
                  <prosody pitch="x-high"> а могу говорить тоном выше </prosody>,
                  или <prosody pitch="x-low">наоборот, ниже</prosody>.
                  Пот+ом, если повезет – <prosody rate="fast">я могу говорить и довольно быстро.</prosody>
                  А еще я умею делать паузы любой длины, например, две секунды <break time="2000ms"/>.
                  <p>
                    Также я умею делать паузы между параграфами.
                  </p>
                  <p>
                    <s>И также я умею делать паузы между предложениями</s>
                    <s>Вот например как сейчас</s>
                  </p>
              </p>
</speak>
'''

audio = model_tts.apply_tts(text=example_text,
                        speaker=speaker,
                        sample_rate=sample_rate,
                        put_accent=put_accent,
                        put_yo=put_yo)
print(example_text)
audio_display = Audio(audio, rate=sample_rate)
display(audio_display)



<speak>
              <p>
                  Когда я просыпаюсь, <prosody rate="x-slow">я говорю довольно медленно</prosody>.
                  Пот+ом я начинаю говорить своим обычным голосом,
                  <prosody pitch="x-high"> а могу говорить тоном выше </prosody>,
                  или <prosody pitch="x-low">наоборот, ниже</prosody>.
                  Пот+ом, если повезет – <prosody rate="fast">я могу говорить и довольно быстро.</prosody>
                  А еще я умею делать паузы любой длины, например, две секунды <break time="2000ms"/>.
                  <p>
                    Также я умею делать паузы между параграфами.
                  </p>
                  <p>
                    <s>И также я умею делать паузы между предложениями</s>
                    <s>Вот например как сейчас</s>
                  </p>
              </p>
</speak>



In [9]:
with open('test.wav', 'wb') as f:
    f.write(audio_display.data)

In [10]:
from huggingsound import SpeechRecognitionModel

model_w2v = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-russian")
audio_paths = ["test.wav"]

transcriptions = model_w2v.transcribe(audio_paths)

01/12/2023 15:08:28 - INFO - huggingsound.speech_recognition.model - Loading model...


100%|██████████| 1/1 [00:06<00:00,  6.31s/it]


In [11]:
raw_recognized =  transcriptions[0]['transcription']
raw_recognized

'когда я проступая все говорю довольно медленно потом я начинаю говорить своим обычным голосом а могу говорить тоном выше или наоборот ниже потом если повезет я могу говорить ей довольно быстро а еще умею делать пауза любоедлины например две секунды также я умею делать паузы между параграфами и также  уме делать палза между предложениями вот например как сейчас'